### Web Scrapping

In [ ]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [4]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [101]:
lang = "indonesian"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "al hijr"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-15"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [102]:
HTMLFile = open(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

content = HTMLFile.read()

soup = BeautifulSoup(content, 'lxml')

ayah_word = soup.findAll('span', attrs={"class":"single-word"})

word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    #arabic = var1.get_text()
    #    arabic = var1.string
    #except:
    #    arabic = ""
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
df_word_list

,arabic,transliterate,translate_indonesian
0,الٓرۚ,alif-lam-ra,Alif Laam Raa
1,تِلۡكَ,til'ka,itu
2,ءَايَٰتُ,ayatu,ayat-ayat
3,ٱلۡكِتَٰبِ,al-kitabi,Al Kitab
4,وَقُرۡءَانٖ,waqur'anin,dan Al Qur'an
...,...,...,...
650,وَٱعۡبُدۡ,wa-u'bud,dan sembahlah
651,رَبَّكَ,rabbaka,Tuhanmu
652,حَتَّىٰ,hatta,sehingga
653,يَأۡتِيَكَ,yatiyaka,datang kepadamu


In [103]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [104]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Al hijr_Arabic_Transliterate_Translate_Words_Bangla.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'Al hijr_Arabic_Transliterate_Translate_Words_Indonesian.xlsx']

In [105]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,الٓرۚ,alif-lam-ra,Alif Laam Ra,الر,अलिफ़ लाम रा,Alif Laam Raa,আলিফ লা-ম রা-,Elif Lam Ra,Алиф лам ра
1,تِلۡكَ,til'ka,These,یہ,ये,itu,এগুলো,şunlar,Это –
2,ءَايَٰتُ,ayatu,(are) the Verses,آیات ہیں,आयात हैं,ayat-ayat,আয়াত,ayetleridir,аяты
3,ٱلۡكِتَٰبِ,al-kitabi,(of) the Book,کتاب کی,किताब की,Al Kitab,কিতাবের,Kitabın,Книги
4,وَقُرۡءَانٖ,waqur'anin,and Quran,اور قرآن,*,dan Al Qur'an,এবং কুরআনের,ve Kur'an'ın,и Корана
...,...,...,...,...,...,...,...,...,...
650,وَٱعۡبُدۡ,wa-u'bud,And worship,اور عبادت کیجئے,और इबादत कीजिए,dan sembahlah,ও ইবাদত করো,ve kulluk et,и поклоняйся
651,رَبَّكَ,rabbaka,your Lord,اپنے رب کی,अपने रब की,Tuhanmu,তোমার রবের,Rabbine,"Господу твоему,"
652,حَتَّىٰ,hatta,until,یہاں تک کہ,यहाँ तक कि,sehingga,যতক্ষণ না,kadar,пока не
653,يَأۡتِيَكَ,yatiyaka,comes to you,آجائے آپ کے پاس,आजाए आपके पास,datang kepadamu,তোমার কাছে আসে,sana gelinceye,придёт к тебе


In [106]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [11]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [12]:
df_faatir = pd.read_excel("/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,ٱلۡحَمۡدُ,al-hamdu,All praises,سب تعریف,सब तारीफ़,segala puji,সকল প্রশংসা,hamd olsun,Хвала
1,لِلَّهِ,lillahi,(be) to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,Allah'a,"Аллаху,"
2,فَاطِرِ,fatiri,Creator,پیدا کرنے والا ہے,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,yoktan var eden,Творцу
3,ٱلسَّمَٰوَٰتِ,al-samawati,(of) the heavens,آسمانوں کا,आसमानों,langit(jamak),আকাশের,gökleri,небес
4,وَٱلۡأَرۡضِ,wal-ardi,and the earth,اور زمین کا,और ज़मीन का,dan bumi,ও পৃথিবীর,ve yeri,"и земли,"
...,...,...,...,...,...,...,...,...,...
770,فَإِنَّ,fa-inna,then indeed,تو بیشک,तो बेशक,maka sesungguh-Nya,নিশ্চযই় তখন,kuşkusuz,"то, поистине,"
771,ٱللَّهَ,al-laha,Allah,اللہ تعالیٰ,अल्लाह,Allah,আল্লাহ,Allah,Аллах
772,كَانَ,kana,is,ہے,है,adalah,হবেন,kullarını,является
773,بِعِبَادِهِۦ,bi'ibadihi,of His slaves,اپنے بندوں کے ساتھ,अपने बन्दों को,dengan hamba-hamba-Nya,তাঁর দাসদের সম্পর্কে,*,Своих рабов


In [13]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,35,Fatir,ٱلۡحَمۡدُ,al-hamdu,All praises,سب تعریف,सब तारीफ़,segala puji,সকল প্রশংসা,hamd olsun,Хвала,56320.0
1,35,Fatir,لِلَّهِ,lillahi,(be) to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,Allah'a,"Аллаху,",56321.0
2,35,Fatir,فَاطِرِ,fatiri,Creator,پیدا کرنے والا ہے,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,yoktan var eden,Творцу,56322.0
3,35,Fatir,ٱلسَّمَٰوَٰتِ,al-samawati,(of) the heavens,آسمانوں کا,आसमानों,langit(jamak),আকাশের,gökleri,небес,56323.0
4,35,Fatir,وَٱلۡأَرۡضِ,wal-ardi,and the earth,اور زمین کا,और ज़मीन का,dan bumi,ও পৃথিবীর,ve yeri,"и земли,",56324.0
...,...,...,...,...,...,...,...,...,...,...,...,...
770,35,Fatir,فَإِنَّ,fa-inna,then indeed,تو بیشک,तो बेशक,maka sesungguh-Nya,নিশ্চযই় তখন,kuşkusuz,"то, поистине,",57090.0
771,35,Fatir,ٱللَّهَ,al-laha,Allah,اللہ تعالیٰ,अल्लाह,Allah,আল্লাহ,Allah,Аллах,57091.0
772,35,Fatir,كَانَ,kana,is,ہے,है,adalah,হবেন,kullarını,является,57092.0
773,35,Fatir,بِعِبَادِهِۦ,bi'ibadihi,of His slaves,اپنے بندوں کے ساتھ,अपने बन्दों को,dengan hamba-hamba-Nya,তাঁর দাসদের সম্পর্কে,*,Своих рабов,57093.0


#### For Loop Adding Info

In [8]:
surah = "an naas"

In [9]:
file = glob.glob(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/*/{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
file

['/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/An Naas-114/An naas_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx']

In [11]:
pd.read_excel(file[0])

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,قُلۡ,qul,Say,کہہ دیجئے,कह दीजिए,katakanlah,বলো,de ki,Скажи:
1,أَعُوذُ,a'udhu,I seek refuge,میں پناہ چاہتا ہوں,मैं पनाह लेता हूँ,aku berlindung,"""আমি আশ্রয় চাই",sığınırım ben,«Я обращаюсь за защитой
2,بِرَبِّ,birabbi,in (the) Lord,رب کی,रब की,Tuhannya,রবের নিকট,Rabbine,к Господу
3,ٱلنَّاسِ,al-nasi,(of) mankind,انسانوں کے,इन्सानों के,manusia,মানুষের,insanların,"людей,"
4,مَلِكِ,maliki,(The) King,بادشاہ,बादशाह की,Rajanya,মহা অধিপতির (নিকট),padişahına,Владыке
5,ٱلنَّاسِ,al-nasi,(of) mankind,انسانوں کے,इन्सानों के,manusia,মানুষের,insanların,"людей,"
6,إِلَٰهِ,ilahi,(The) God,الٰہ کی,इलाह की,Tuhannya,ইলাহ্‌র (নিকট),tanrısına,Богу
7,ٱلنَّاسِ,al-nasi,(of) mankind,انسانوں کے,इन्सानों के,manusia,মানুষের,insanların,"людей,"
8,مِن,min,From,سے,*,dari,হতে,şerrinden,от
9,شَرِّ,sharri,(the) evil,شر,शर से,kejahatan,অমঙ্গল,*,зла
